Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [X] Continue to clean and explore your data. 
- [X] For the evaluation metric you chose, what score would you get just by guessing?
- [X] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

Your code starts here:

## **Data Exploration**

In [4]:
# Imports
import pandas as pd
import numpy as np

# Create dataframe of project data
potholes = pd.read_csv('../../Unit 2 Build Week/Pothole Requests after 10.15.18.csv')

In [13]:
# Change output viewing options
pd.set_option('display.max_columns', 50)

# Look at data
potholes.head()

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION TIME,CREATION MONTH,CREATION YEAR,STATUS,EXCEEDED EST TIMEFRAME,CLOSED DATE,CLOSED MONTH,CLOSED YEAR,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNTY,COUNCIL DISTRICT,POLICE DISTRICT,PARCEL ID NO,LATITUDE,LONGITUDE,CASE URL,30-60-90 Days Open Window
0,2019169222,PHONE,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,Streets / Roadways / Alleys,Pothole,District 3,09/20/2019,01:31 PM,9,2019,RESOL,Y,11/07/2019,11.0,2019.0,48.0,13110 E 57th St,"13110 E 57th St64133\n(39.018605, -94.426046)",64133,Blue Vue Hills,Jackson,5,East,63787,39.018605,-94.426046,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN
1,2019126105,PHONE,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,Streets / Roadways / Alleys,Pothole,District 3,07/03/2019,08:30 AM,7,2019,RESOL,Y,10/24/2019,10.0,2019.0,113.0,1420 E 75th Ter,"1420 E 75th Ter64131\n(38.990601, -94.570141)",64131,East Meyer 6,Jackson,5,Metro,115894,38.990601,-94.570141,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN
2,2019126468,WEB,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,Streets / Roadways / Alleys,Pothole,District 3,07/03/2019,01:53 PM,7,2019,RESOL,Y,10/24/2019,10.0,2019.0,113.0,5500 Michigan Ave,"5500 Michigan Ave64130\n(39.026622, -94.563667)",64130,Blue Hills,Jackson,5,Metro,147750,39.026622,-94.563667,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN
3,2019183619,PHONE,Public Works,Public Works-Street and Traffic-District 1,Streets / Roadways / Alleys-Pothole-District 1,Streets / Roadways / Alleys,Pothole,District 1,10/23/2019,09:36 AM,10,2019,RESOL,Y,11/22/2019,11.0,2019.0,30.0,6344 NE Pleasant Valley Rd,"6344 NE Pleasant Valley Rd64119\n(39.216929, -...",64119,Shoal Creek,Clay,1,Shoal Creek,89139,39.216929,-94.504961,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN
4,2019190590,PHONE,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,Streets / Roadways / Alleys,Pothole,District 3,11/08/2019,10:27 AM,11,2019,RESOL,Y,11/18/2019,11.0,2019.0,10.0,9500 Blue Ridge Blvd,"9500 Blue Ridge Blvd64134\n(38.952042, -94.508...",64134,Fairlane,Jackson,5,South,56861,38.952042,-94.508463,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN


In [52]:
potholes.describe(include='all')

,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CATEGORY,TYPE,DETAIL,CREATION DATE,CREATION TIME,CREATION MONTH,CREATION YEAR,STATUS,EXCEEDED EST TIMEFRAME,CLOSED DATE,CLOSED MONTH,CLOSED YEAR,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNTY,COUNCIL DISTRICT,POLICE DISTRICT,PARCEL ID NO,LATITUDE,LONGITUDE,CASE URL,30-60-90 Days Open Window
count,1.958200e+04,19582,19582,19582,19582,19582,19582,19582,19582,19582,19582.000000,19582.000000,19582,19582,19163,19161.000000,19161.000000,19161.000000,19582,19582,19582.000000,19236,19582,19582.000000,19582,19582.000000,19582.000000,19582.000000,19582,419.000000
unique,NaN,9,1,3,4,1,1,4,412,1229,NaN,NaN,4,2,308,NaN,NaN,NaN,10369,10368,NaN,240,4,NaN,6,NaN,NaN,NaN,19582,NaN
top,NaN,WEB,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,Streets / Roadways / Alleys,Pothole,District 3,02/25/2019,09:12 AM,NaN,NaN,RESOL,Y,07/31/2019,NaN,NaN,NaN,9555 N Oak Trfy,"9555 N Oak Trfy64155\n(39.266923, -94.574397)",NaN,Shoal Creek,Jackson,NaN,Metro,NaN,NaN,NaN,http://city.kcmo.org/kc/ActionCenterRequest/Ca...,NaN
freq,NaN,8685,19582,10925,10925,19582,19582,10925,248,45,NaN,NaN,19084,16429,495,NaN,NaN,NaN,49,49,NaN,867,15379,NaN,6567,NaN,NaN,NaN,1,NaN
mean,2.019054e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.950158,2018.964661,NaN,NaN,NaN,6.072282,2018.966651,38.967695,NaN,NaN,64127.450363,NaN,NaN,4.207078,NaN,119251.633490,39.054155,-94.560354,NaN,1.073986
std,1.772217e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.686897,0.184639,NaN,NaN,NaN,2.647121,0.179551,36.931906,NaN,NaN,15.156395,NaN,NaN,1.630698,NaN,66335.365439,0.111661,0.084290,NaN,6.954518
min,2.018139e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2018.000000,NaN,NaN,NaN,1.000000,2018.000000,0.000000,NaN,NaN,64030.000000,NaN,NaN,1.000000,NaN,7.000000,35.797519,-103.890579,NaN,0.000000
25%,2.019050e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2019.000000,NaN,NaN,NaN,4.000000,2019.000000,9.000000,NaN,NaN,64114.000000,NaN,NaN,3.000000,NaN,66954.750000,38.977592,-94.594010,NaN,0.000000
50%,2.019079e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,2019.000000,NaN,NaN,NaN,6.000000,2019.000000,27.000000,NaN,NaN,64130.000000,NaN,NaN,5.000000,NaN,116784.500000,39.026663,-94.574313,NaN,0.000000
75%,2.019114e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,2019.000000,NaN,NaN,NaN,8.000000,2019.000000,61.000000,NaN,NaN,64134.000000,NaN,NaN,6.000000,NaN,155151.250000,39.104756,-94.531724,NaN,0.000000


In [68]:
potholes[potholes['STATUS'] == 'RESOL'].isna().shape

(19084, 30)

In [57]:
potholes.columns

Index(['CASE ID', 'SOURCE', 'DEPARTMENT', 'WORK GROUP', 'REQUEST TYPE',
       'CATEGORY', 'TYPE', 'DETAIL', 'CREATION DATE', 'CREATION TIME',
       'CREATION MONTH', 'CREATION YEAR', 'STATUS', 'EXCEEDED EST TIMEFRAME',
       'CLOSED DATE', 'CLOSED MONTH', 'CLOSED YEAR', 'DAYS TO CLOSE',
       'STREET ADDRESS', 'ADDRESS WITH GEOCODE', 'ZIP CODE', 'NEIGHBORHOOD',
       'COUNTY', 'COUNCIL DISTRICT', 'POLICE DISTRICT', 'PARCEL ID NO',
       'LATITUDE', 'LONGITUDE', 'CASE URL', '30-60-90 Days Open Window'],
      dtype='object')

In [35]:
# Look at various value_counts 
potholes['WORK GROUP'].value_counts()

Public Works-Street and Traffic-District 3    10925
Public Works-Street and Traffic-District 2     4449
Public Works-Street and Traffic-District 1     4208
Name: WORK GROUP, dtype: int64

In [36]:
potholes['DETAIL'].value_counts()

District 3    10925
District 2     4408
District 1     4208
Bridge           41
Name: DETAIL, dtype: int64

### Notes on completed exploration of each column:

Case ID: DROP - at least for model (but ensure they are all unique)  
Source: keep (9 values) 
Department: DROP - constant  
Work Group: keep? (3 values)  
Request Type: DROP - same as Detail  
Category: DROP - constant  
Type: DROP - constant  
Detail:  drop either Detail or Work Group (detail includes ‘bridge’) (4 values)  
Creation Date: Convert to date-time  
Creation Time: Bin? May be a factor, may not be  
Creation Month: ??? remove? if keeping ‘date’  
Creation Year: ??? remove? if keeping ‘date’  
Status: Clean? (may not need to after cleaning target column) — that should drop Open & Cancelled. How to handle duplicate? Create new column? (4 values)  
Exceeded East Timeframe: Drop? Probably includes leakage — info added after case resolved? (Y/N)  
Closed Date: Drop? no new info not gained through “days to close” ?  
Closed Month: Drop? no new info not gained through “days to close” ?  
Closed Year: Drop? no new info not gained through “days to close” ? 
Days to Close: TARGET  
Street Address: Use to engineer another feature? About street usage? About duplicates? (there are duplicate addresses!)  
Address With Geocode: DROP — combination of street address and lat/lon  
ZIP CODE: feature — keep? but keep in mind other area indicators  
Neighborhood: feature? (240 values) — may need to drop due to high cardinality   
County: feature? (4 values)  
Council District: keep? (overlap w/ county, police, or neighborhood?)  
Police District: feature? (6 values)  
Parcel ID No: what is this ?? drop?   
Latitude: keep  
Longitude: keep  
Case URL: DROP - unhelpful for model, and case can be searched w/ ID# — all unique  
30-60-90 Days Open Window: DROP - flag only on currently open cases (use to create any new features?)  

In [70]:
# Convert CREATION DATE to datetime to prepare for data split
potholes['CREATION DATE'] = pd.to_datetime(potholes['CREATION DATE'])


In [78]:
# Split data into train, validate, and test sets

# Set train to Oct 15, 2018 (first freeze of 2018) - June 14, 2019
train = potholes[potholes['CREATION DATE'] < '2019-06-15']

# Set val to the next two months of the dataset
val = potholes[(potholes['CREATION DATE'] >= '2019-06-15') 
               & (potholes['CREATION DATE'] < '2019-08-15')]

# Set test to the last two months of the year following start date
test = potholes[(potholes['CREATION DATE'] >= '2019-08-15')
               & (potholes['CREATION DATE'] < '2019-10-15')]

# print shape of remaining data to confirm correct splitting
print(potholes[potholes['CREATION DATE'] >= '2019-10-15'].shape)

train.shape, val.shape, test.shape

(825, 30)


((14864, 30), (2625, 30), (1268, 30))

In [79]:
# print shape of original df to confirm correct shapes of split dfs
potholes.shape

(19582, 30)

#### For the evaluation metric you chose, what score would you get just by guessing?


In [86]:
# Find mean baseline for train
target = 'DAYS TO CLOSE'
baseline = train[target].mean()
print(f'Baseline Mean: {baseline:.2f} days')

# Find baseline MAE
from sklearn.metrics import mean_absolute_error
y_train = train[target].dropna()
y_pred = [baseline] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Baseline Mean Absolute Error: {mae:.2f} days')
print('Baseline R^2: 0')

Baseline Mean: 40.96 days
Baseline Mean Absolute Error: 32.56 days
Baseline R^2: 0


#### Can you make a fast, first model that beats guessing?


In [115]:
def wrangle(X):
    
    '''Wrangles train, validate, and test sets in the same way'''
    
    # Avoid SettingWithCopyWarning
    X = X.copy()
    
    # Use only 'Resolved' cases
    X = X[X['STATUS'] == 'RESOL']
    
    # Ensure no remaining null values in target column
    X = X.dropna(subset=['DAYS TO CLOSE'])
    
    # Use only rows with greater than 0 days to close
    X = X[X['DAYS TO CLOSE'] > 0]
    
    # Drop columns with constant values
    constant = ['DEPARTMENT', 'CATEGORY', 'TYPE']
    X = X.drop(columns=constant)
    
    # Drop columns with duplicate values
    duplicate = ['REQUEST TYPE', 'ADDRESS WITH GEOCODE']
    X = X.drop(columns=duplicate)
    
    # Drop column at risk for leakage
    leakage = ['EXCEEDED EST TIMEFRAME', '30-60-90 Days Open Window']
    X = X.drop(columns=leakage)
    
    # Drop high cardinality columns
    cardinality = ['CASE ID', 'STREET ADDRESS', 'NEIGHBORHOOD', 
                   'PARCEL ID NO', 'CASE URL', 'CREATION DATE', 'CLOSED DATE']
    X = X.drop(columns=cardinality)
    
    # Convert ZIP CODE to categorical variable
    X['ZIP CODE'] = X['ZIP CODE'].astype('category')
        
    return X

train_wrangled = wrangle(train)
val_wrangled = wrangle(val)
test_wrangled = wrangle(test)

In [119]:
train_wrangled.columns

Index(['SOURCE', 'WORK GROUP', 'DETAIL', 'CREATION TIME', 'CREATION MONTH',
       'CREATION YEAR', 'STATUS', 'CLOSED MONTH', 'CLOSED YEAR',
       'DAYS TO CLOSE', 'ZIP CODE', 'COUNTY', 'COUNCIL DISTRICT',
       'POLICE DISTRICT', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [116]:
# Arrange data into X features matrix and y target vector
target = 'DAYS TO CLOSE'
X_train = train_wrangled.drop(columns=target)
y_train = train_wrangled[target]
X_val = val_wrangled.drop(columns=target)
y_val = val_wrangled[target]
X_test = test_wrangled.drop(columns=target)
y_test = test_wrangled[target]

In [129]:
# Imports for pipeline
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.tree import DecisionTreeRegressor

# Create pipeline
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    DecisionTreeRegressor(min_samples_leaf=20, random_state=7)
    
)

# Fit on train df
pipeline.fit(X_train, y_train);

In [132]:
# Find evaluation metrics
from sklearn.metrics import mean_absolute_error

print('Train R^2', pipeline.score(X_train, y_train))
prediction = pipeline.predict(X_train)
mae = mean_absolute_error(y_train, prediction)
print(f'Train Mean Absolute Error: {mae:.2f} days\n')

print('Validation R^2', pipeline.score(X_val, y_val))
prediction = pipeline.predict(X_val)
mae = mean_absolute_error(y_val, prediction)
print(f'Validation Mean Absolute Error: {mae:.2f} days')

Train R^2 0.955333652947819
Train Mean Absolute Error: 6.12 days

Validation R^2 -1.1315103175489285
Validation Mean Absolute Error: 31.60 days


In [ ]:
# From this very baseline model, I have dropped my validation MAE by less than
# one day from baseline and gotten a negative R^2 value... nowhere to go but up!